Curve Fitting
=============

In [10]:
(require '[provisdom.veja.core :as veja]
         '[provisdom.veja-charts.core :as charts]
         '[provisdom.math.core :as m]
         '[provisdom.math.combinatorics :as combo]
         '[provisdom.math.series :as series]
         '[provisdom.solvers.curve-fitting :as curve-fitting])

## Line Fitting

We are going to find a line to best fit a small number of points.

In [11]:
(defn point-data
    [points-to-show]
    (map (fn [[x y]] 
             {:Actual-X x 
              :Actual-Y y
              :Actual-Type :points}) 
         points-to-show))

(defn linear-least-squares-data-fn
    [[basis-fn basis-fn-name] x-chart-values points]
    (let [lls-fn (curve-fitting/linear-least-squares-line-fitting
                     {::curve-fitting/x-vals   (mapv first points)
                      ::curve-fitting/f-vals   (mapv second points)
                      ::curve-fitting/basis-fn basis-fn})]
        (map (fn [x] 
                 {:X x 
                  :Y ((::curve-fitting/line-fitting-fn lls-fn) x) 
                  :Type basis-fn-name})
             x-chart-values)))

(defn b-spline-data-fn
    [degree x-chart-values points]
    (let [b-spline-fn (curve-fitting/b-spline-line-fitting
                     {::curve-fitting/distinct-x-vals (mapv first points)
                      ::curve-fitting/f-vals          (mapv second points)
                      ::curve-fitting/degree          degree})]
        (map (fn [x] 
                 {:X x 
                  :Y (b-spline-fn x) 
                  :Type (str "b-spline degree " degree)})
             x-chart-values)))

(defn data-values
    [x-chart-values points points-to-show basis-fn-with-names b-spline-degrees]
    (vec (concat (point-data points-to-show) 
                 (mapcat (fn [basis-fn-with-name] 
                             (linear-least-squares-data-fn basis-fn-with-name x-chart-values points)) 
                         basis-fn-with-names)
                 (mapcat (fn [degree] 
                             (b-spline-data-fn degree x-chart-values points)) 
                         b-spline-degrees))))

#'user/data-values

Initially, we will compare a number of different chebyshev of the 1st kind basis functions in the `linear-least-squares-line-fitting` function. The dots represent the points. As expected, the higher the degree, the closer the fit, until the 4th degree is able to match the points.

In [12]:
(def points-data [[2.0 26.0] [4.0 173.0] [5.0 281.0] [6.0 322.0] [7.0 37.0]])

(def x-chart-range (range 1.0 7.25 0.01))

(def chebyshev-1st-kind-basis-fns-with-names
    [[(series/polynomial-fn 4 {::series/chebyshev-kind 1}) 
      "chebyshev 1st-kind 4th degree"]
     [(series/polynomial-fn 3 {::series/chebyshev-kind 1}) 
      "chebyshev 1st-kind 3rd degree"]
     [(series/polynomial-fn 2 {::series/chebyshev-kind 1}) 
      "chebyshev 1st-kind 2nd degree"]
     [(series/polynomial-fn 1 {::series/chebyshev-kind 1}) 
      "chebyshev 1st-kind 1st degree"]])

(def multi-line-data
    (data-values x-chart-range points-data points-data chebyshev-1st-kind-basis-fns-with-names []))

(def line-chart-map 
    {:x :X
     :y :Y
     :series :Type
     :x-pt :Actual-X
     :y-pt :Actual-Y
     :series-pt :Actual-Type
     :width 800
     :height 500})

(veja/vega-lite 
    (charts/multi-line-chart-with-points 
        (assoc line-chart-map :data multi-line-data)))

Next, we can look at a variety of other basis functions.

In [13]:
(def variety-of-basis-fns-with-names
    [[(series/polynomial-fn 4 {::series/chebyshev-kind 2}) 
      "chebyshev 2nd-kind 4th degree"]
     [(series/polynomial-fn 3) 
      "polynomial 3rd degree"]
     [(series/polynomial-fn 5 {::series/start-degree 3}) 
      "polynomial 3rd->5th degree"]])

(def multi-line-data2 
    (data-values x-chart-range points-data points-data variety-of-basis-fns-with-names []))

(veja/vega-lite 
    (charts/multi-line-chart-with-points 
        (assoc line-chart-map :data multi-line-data2)))

Alternatively, we can use the `b-spline-line-fitting` function to use B-splines as our basis. Higher-order basis functions match degree 3 in this example.

In [14]:
(def b-spline-degrees [1 2 3])
(def multi-line-data3
    (data-values x-chart-range points-data points-data [] b-spline-degrees))

(veja/vega-lite 
    (charts/multi-line-chart-with-points 
        (assoc line-chart-map :data multi-line-data3)))

## Path Fitting

We are going to find a best fit of a path to points over a single "time" dimension, `t`.

In [15]:
(defn- path-fn
    [t-points basis-fn]
    (fn [t]
        (map (fn [lls-fn]
                 ((::curve-fitting/line-fitting-fn lls-fn) t))
             (map (fn [pts] 
                      (curve-fitting/linear-least-squares-line-fitting 
                          {::curve-fitting/x-vals   (mapv first t-points)
                           ::curve-fitting/f-vals   pts
                           ::curve-fitting/basis-fn basis-fn}))
                  (let [tpts (mapv second t-points)]
                      [(mapv first tpts) (mapv second tpts)])))))

(defn path-points
    [t-points]
    (mapv (fn [[t [xt yt]]] 
              {:Actual-X xt 
               :Actual-Y yt
               :Actual-Type :points
               :actual-index t}) 
          t-points))

(defn path-data
    [t-chart-values t-points [basis-fn basis-fn-name]]
    (let [p-fn (path-fn t-points basis-fn)]
        (mapv (fn [t]
                  (let [[x y] (p-fn t)]
                      {:X x 
                       :Y y
                       :Type basis-fn-name 
                       :index t}))
              t-chart-values)))

(defn path-data-values
    [t-chart-values t-points t-points-to-show basis-fns-with-names]
    (concat (path-points t-points-to-show)
            (mapcat (fn [basis-fn-with-name] 
                        (path-data t-chart-values t-points basis-fn-with-name))
                    basis-fns-with-names)))

#'user/path-data-values

Any of the line-fitting functions should work here, so we'll just try a variety of linear least squares functions.

In [16]:
(def t-points-data [[2.0 [26.0 34.0]] [4.0 [173.0 62.0]] [5.0 [281.0 71.0]] [6.0 [322.0 79.0]] [7.0 [37.0 86.0]]])

(def parametric-data (path-data-values x-chart-range t-points-data t-points-data variety-of-basis-fns-with-names))

(def parametric-plot-map 
    {:x :X
     :y :Y
     :series :Type
     :order :index
     :x-pt :Actual-X
     :y-pt :Actual-Y
     :series-pt :Actual-Type
     :order-pt :actual-index
     :width 800
     :height 500})

(veja/vega-lite 
    (charts/multi-parametric-plot-with-points 
        (assoc parametric-plot-map :data parametric-data)))

## 2D Curve Fitting

We are going to find the best fit to points in 2D. 

In [17]:
(defn curve-fitting-2D-fn 
    [basis-2D-fn x-chart-values y-chart-values points-2D]
    (let [curve-fn (curve-fitting/linear-least-squares-curve-fitting 
                   {::curve-fitting/x-matrix       (mapv first points-2D)
                    ::curve-fitting/f-vals         (mapv second points-2D)
                    ::curve-fitting/curve-basis-fn (fn [v]
                                                       (if (= 2 (count v))
                                                           (let [[x y] v
                                                                 val-v (basis-2D-fn x y)]
                                                               val-v)
                                                           []))})]
        (mapcat (fn [x]
                    (map (fn [y]
                             {:X x 
                              :Y y 
                              :Z ((::curve-fitting/curve-fitting-fn curve-fn) [x y])})
                         y-chart-values)) 
                x-chart-values)))

#'user/curve-fitting-2D-fn

Below is a heatmap of best fit for `points-in-2D`. We're using `(series/polynomial-2D-fn-by-degree 2)` as the basis function.

In [18]:
(def points-in-2D
  [[[2.0 18.0] 796.0]
   [[2.0 20.0] 1090.0]
   [[2.0 21.0] 1261.0]
   [[2.0 23.0] 1654.0]
   [[2.0 26.0] 2387.0]
   [[2.0 45.0] 12344.0]
   [[4.0 18.0] 114.0]
   [[4.0 20.0] 153.0]
   [[4.0 21.0] 177.0]
   [[4.0 23.0] 230.0]
   [[4.0 26.0] 329.0]
   [[4.0 45.0] 1677.0]
   [[5.0 18.0] 47.0]
   [[5.0 20.0] 62.0]
   [[5.0 21.0] 70.0]
   [[5.0 23.0] 90.0]
   [[5.0 26.0] 126.0]
   [[5.0 45.0] 623.0]
   [[6.0 18.0] 23.0]
   [[6.0 20.0] 28.0]
   [[6.0 21.0] 32.0]
   [[6.0 23.0] 39.0]
   [[6.0 26.0] 52.0]
   [[6.0 45.0] 235.0]
   [[7.0 18.0] 15.0]
   [[7.0 20.0] 17.0]
   [[7.0 21.0] 18.0]
   [[7.0 23.0] 21.0]
   [[7.0 26.0] 26.0]
   [[7.0 45.0] 93.0]])

(def chart-x-range (range 1.0 7.25 0.02))
(def chart-y-range (range 14.0 49.0 0.2))
(def x-count (* 6.25 50))
(def y-count (/ 35 0.2))

(veja/vega 
    (charts/heat-map 
        {:data (curve-fitting-2D-fn (series/polynomial-2D-fn-by-degree 2) chart-x-range chart-y-range points-in-2D) 
        :x :X 
        :y :Y 
        :z :Z
        :width 750
        :height 500
        :x-count x-count
        :y-count y-count}))
